In [34]:
from sentence_transformers import SentenceTransformer

from os import listdir
from os.path import isfile, join

import csv
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import requests
import re

In [2]:
cid="715ae75406e24c24a3641929260ca343"
secret="e60a686258c646b7a4d3f9636b3235db"

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [41]:
AUTH_URL = "https://accounts.spotify.com/api/token"
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': cid,
    'client_secret': secret,
})

auth_response_data = auth_response.json()

access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

def getInfo(headers, artistName, trackName):

    trackName = trackName.replace(' ', '%20')
    
    artistName = artistName.replace(' ', '%20')
    
    BASE_URL = 'https://api.spotify.com/v1/'
    r = requests.get(BASE_URL + 'search?q=' + artistName + '%20track:' + trackName + '&type=track', headers=headers)
    r = r.json()

    return r['tracks']['items'][0]

getInfo(headers,'LOCK','New York vs Paris')

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3yhBAMGFmgF1FRFdWAqwJS'},
    'href': 'https://api.spotify.com/v1/artists/3yhBAMGFmgF1FRFdWAqwJS',
    'id': '3yhBAMGFmgF1FRFdWAqwJS',
    'name': 'Lock',
    'type': 'artist',
    'uri': 'spotify:artist:3yhBAMGFmgF1FRFdWAqwJS'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BI',
   'BJ',
   'BN',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BY',
   'BZ',
   'CA',
   'CD',
   'CG',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'GY',
   'HK',
   'HN',
   'HR',
   'HT',
   'HU

In [5]:
mypath = '../../data2.csv'

df = pd.read_csv(mypath)

In [6]:
df.head()

,title,tag,artist,year,views,features,lyrics,id,lang
0,New York vs Paris,pop,LOCK (Band),2017,21,{},"['Paris, New York', 'When everythings wrong in...",3995408,en
1,Addicted,rap,J,2021,22,{},['I think my biggest fear other than dying is ...,6638237,en
2,BONUS Wetter,rap,GV Honcho,2019,131,{JocThaDon},"['Ayyye', 'Honcho', 'I’m in the whip and I’m c...",4491198,en
3,The City Of The Lord,pop,Parachute Band,2012,111,{},"['God our refuge and strength', ""He's our help...",2260781,en
4,You Look So Sad,pop,Rhode Island,2008,34,{},"['Acid thrown from a silver tray', 'Stripping ...",1674621,en


In [51]:
df1k = df.head(1000)

def getUri(row):

    artistName = row['artist']
    trackName = row['title']

    artistName = re.sub("[\(\[].*?[\)\]]", "", artistName)
    trackName = re.sub("[\(\[].*?[\)\]]", "", trackName)

    artistName = artistName.strip()
    trackName = trackName.strip()

    trackName2 = trackName.replace(' ', '%20')
    artistName2 = artistName.replace(' ', '%20')

    BASE_URL = 'https://api.spotify.com/v1/'
    r = requests.get(BASE_URL + 'search?q=' + artistName2 + '%20track:' + trackName2 + '&type=track', headers=headers)
    r = r.json()

    if r['tracks']['items'] == []:
        print('\nno search results found:',artistName,'|',trackName)
        return ''

    if r['tracks']['items'][0]['name'].upper() == trackName.upper() and r['tracks']['items'][0]['artists'][0]['name'].upper() == artistName.upper():
        print('\ngottem:',artistName,'|',trackName)
        return r['tracks']['items'][0]['uri']
    
    else:
        for item in r['tracks']['items']:
            if item['name'].upper() == trackName.upper() and item['artists'][0]['name'].upper() == artistName.upper():
                print('\ngottem:',artistName,'|',trackName)
                return item['uri']
            
        print('\nnone of the items matched:',artistName,'|',trackName)
        return ''
    
    

In [52]:
df1k['uri'] = df1k.apply(lambda row : getUri(row), axis = 1)


no search results found: GV Honcho | BONUS Wetter

no search results found: Lil Grim GARZO | KARMA

no search results found: Bradleyd | Came Back

no search results found: Meat Loaf | Lost Love

no search results found: Rich $tunna | For The Muny

no search results found: Jaden Gabb | Sad to Say Things Dont Change

no search results found: Rab Neba | ISAA

no search results found: Smhrevenge | Restrictions: I can’t love you the way I want to

no search results found: Abyssaria | Forgotten Worlds

no search results found: De | Cold Side of My Pillow

no search results found: Jem | And So I Pray Starry Sky Acoustic Mix

no search results found: Aerosol Airplanes | No I Am Your Father

no search results found: Joshua Radin | The One You Knew

no search results found: Fishplate | Trashcatcher

no search results found: Crumbcher | Here I Am

no search results found: 2gaudy | We Cant Do This Anymore We Already Know

no search results found: The Almighty RSO | Forever RSO

no search results 


no search results found: Bonnie "Prince" Billy | Cows

no search results found: George Young | Wipe Me Down Freestyle

no search results found: Connor Quest! | Trap Card 2 Contest Entry

no search results found: Jason Mraz | Did you get my message? - live from shubas

no search results found: Christine Guldbrandsen | In Your Embrace

no search results found: Joelle | Im Done

no search results found: De La Soul | I Can Do Anything Delacratic

no search results found: Evanescence | Lies

no search results found: Uriah Heep | Lost One Love

no search results found: Phantom Tex | Occultic Master

no search results found: Cody Vondell | Yellow Bricks End Remix

no search results found: Ellis Martin | Hear Me Out PART 2

no search results found: Lorenzo Laflame | Faded out.

no search results found: rocket // dejade-javu | 1 in 8 freestyle

no search results found: Rupesh Cartel | About Denial Rename club mix

no search results found: ExpediteOfficial | Puzzle

no search results found: SL5


no search results found: 2Pac | Bad Guy Dj Mimo Remix

no search results found: The Effex | Don’t Take Her Away

no search results found: Ripsi WIneLover | Beez In The Trap

no search results found: XXXHXAILO | Vengeance/Never

no search results found: Reggie D | Maame : The Story  Ay3 late cover

no search results found: Vigilantes of Love | Paralyzed

no search results found: Vinylshakerz | Rainbow - Radio Edit

no search results found: Fuckedup_alt | Anime

no search results found: Omen | Phone Home

no search results found: STK 23 | Can’t Trust

no search results found: Neil Young | Cowgirl In The Sand Live At Massey Hall 1971

no search results found: Josef Michel | Free interlude

no search results found: Smokepurpp | Regrets

no search results found: Genius English Translations | NCT DREAM - 사랑한단 뜻이야 Candle Light English Translation

no search results found: Justonho | Hyperbolic Time Chamber Prod. Josh Petruccio

no search results found: Above Average | Sushi

no search result

C:\Users\terre\AppData\Local\Temp\ipykernel_18912\2693008118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1k['uri'] = df1k.apply(lambda row : getUri(row), axis = 1)


In [71]:
test = df1k.iloc[2:5].copy(deep=True)

test.head()

,title,tag,artist,year,views,features,lyrics,id,lang,uri
2,BONUS Wetter,rap,GV Honcho,2019,131,{JocThaDon},"['Ayyye', 'Honcho', 'I’m in the whip and I’m c...",4491198,en,
3,The City Of The Lord,pop,Parachute Band,2012,111,{},"['God our refuge and strength', ""He's our help...",2260781,en,spotify:track:4k1sCFf4WrSQjFnWXHXPyD
4,You Look So Sad,pop,Rhode Island,2008,34,{},"['Acid thrown from a silver tray', 'Stripping ...",1674621,en,spotify:track:4Ph0U48DHfKTLeCSip5zGr


In [73]:
def test2(row):
    return 2

test['testA'] = test.apply(lambda row : test2(row), axis = 1)

test.head()

,title,tag,artist,year,views,features,lyrics,id,lang,uri,testA
2,BONUS Wetter,rap,GV Honcho,2019,131,{JocThaDon},"['Ayyye', 'Honcho', 'I’m in the whip and I’m c...",4491198,en,,2
3,The City Of The Lord,pop,Parachute Band,2012,111,{},"['God our refuge and strength', ""He's our help...",2260781,en,spotify:track:4k1sCFf4WrSQjFnWXHXPyD,2
4,You Look So Sad,pop,Rhode Island,2008,34,{},"['Acid thrown from a silver tray', 'Stripping ...",1674621,en,spotify:track:4Ph0U48DHfKTLeCSip5zGr,2
